# V26 — Complete DataFrame Variable Flow Map

**Version**: 26.0.0 | **Port**: 8102 | **Date**: 2026-02-16

This notebook documents every DataFrame variable in the FCDAI pipeline, in sequence, from data source upload through final scored output.

## Phase 1 — Data Sources Page: 12 Tables → Merged Master

### 12 Source Table Slots

| # | Slot Name | Vault Key(s) | Type | Mandatory |
|---|-----------|-------------|------|-----------|
| 1 | **BASE** | `BASE` | Tabular | **YES** |
| 2 | transactions | `transactions` | Tabular | No |
| 3 | customer_party | `customer_party` | Tabular | No |
| 4 | accounts | `accounts` | Tabular | No |
| 5 | alerts | `alerts` | Tabular | No |
| 6 | cases | `cases` | Tabular | No |
| 7 | kyc | `kyc` | Tabular | No |
| 8 | watchlist | `watchlist` | Tabular | No |
| 9 | relationships | `relationships_edges` + `relationships_nodes` | Graph (EXTRACT) | No |
| 10 | temporal | `temporal_long` + `temporal_wide` | Time-series (EXTRACT) | No |
| 11 | others1 | `others1` | Tabular | No |
| 12 | others2 | `others2` | Tabular | No |

**Storage**: Each table saved as `data/vault/sources/{name}.parquet`  
**Loading**: `DataVault.load_sources()` → `dict[str, pd.DataFrame]`

### Merge Sequence (Canonical Path — RollupEngine)

**File**: `layers/rollup_engine.py` → `build_master()` (lines 716-913)

```
Step 1: base_df = sources["BASE"].copy()
        master_df = base_df.drop_duplicates(subset=[cust_id])

Step 2: Scan & classify all 11 optional tables

Step 3: For each TABULAR table (9 tables):
          rolled = rollup_table(table)     ← grain normalization
          master_df = master_df.merge(rolled, on=cust_id, how='left')
          
        Merge Order:
          master_df ← transactions    (suffix _tra)
          master_df ← customer_party  (suffix _cus)
          master_df ← accounts        (suffix _acc)
          master_df ← alerts          (suffix _ale)
          master_df ← cases           (suffix _cas)
          master_df ← kyc             (suffix _kyc)
          master_df ← watchlist       (suffix _wat)
          master_df ← others1         (suffix _oth)  ⚠ collision
          master_df ← others2         (suffix _oth)  ⚠ collision

Step 4: graph_data = extract_graph_data(relationships_edges)
        (NOT joined to master — kept separate)

Step 5: temporal_data = extract_temporal_data(temporal_long)
        (NOT joined to master — kept separate)

Step 6: master_df = apply_exclusions(master_df)  ← remove PII/constants

Step 7: result.master_df = master_df              ← FINAL MASTER
```

**Result**: `MasterBuildResult.master_df` → accessed as `build_result.master_df`

### Variable Handoff: Data Sources → Pipeline

```
data_sources.py:
  df_master = build_result.master_df          # L1023
  data_vault.set_current_data(df_master)       # saves → vault/current.parquet

pipeline_run.py:
  df_master, n_tables, n_customers = _load_aggregate()   # V26: renamed from agg_df
  df_merged = data_vault.get_data()             # loads ← vault/current.parquet
  
  # Fallback merges (if no current.parquet):
  df_base = sources["BASE"].copy()              # start accumulator
  for tname, tdf in sources.items():
      df_base = df_base.merge(tdf, ...)         # loop join (NO rollup!)
  df_merged = df_base                           # rename
  
  # Last resort:
  df_fallback = max(sources.values(), key=len)  # V26: renamed from 'biggest'
  df_merged = df_fallback

  # Pass to pipeline:
  result = pipeline.run(sources=sources, merged_df=df_merged)
```

## Phase 2 — Pipeline Execution: 14-Step Variable Chain

**File**: `pipeline.py` → `ApurbaDasPipeline.run()` (lines 196-640)

| Step | Variable | Layer | Shape (sample) | Producer | Description |
|------|----------|-------|----------------|----------|-------------|
| 1 | `merged_df` | Input | (32148, 76) | vault/current.parquet | Raw BASE+merged table from Data Sources |
| 2 | **`df_raw`** | L1-2 | (32148, 76) | `IngestPipeline.dq.cleanse()` | DQ cleansed: strip whitespace, fill gaps, dedup |
| 3 | *(M1 scorecard)* | M1 | read-only | `DQValidationEngine.validate(df_raw)` | 6-step gate: DQ Score 99.2% PASS |
| 4 | **`df_agg`** | V6 | (100, 32) | `CustomerAggregator.aggregate(df_raw)` | 32K txns → 100 customers. Amount stats, patterns |
| 5 | **`df_features`** | L3 | (100, 52) | `L3FeatureEngineering.engineer_features(df_agg)` | +20 cols: velocity, behavioral, ratio, flags |
| 6 | *(V8 schema)* | V8 | read-only | `SchemaDetector.detect_schema(df_features)` | 41 usable / 52 total columns profiled |
| 7 | **`df_cleaned`** | M2 | (100, 43) | `DQProcessingEngine.process(df_features)` | 12-step cleanse, dropped 9 redundant cols |
| 8 | **`df_encoded`** | M3 | (100, 42) | `PreprocessingEngine.preprocess(df_cleaned)` | 100% float64 matrix. Categorical encoded |
| 9 | *(scaled_matrices)* | M4 | dict(6 keys) | `ScalingEngine.scale(df_encoded)` | 6 scalers: Standard/MinMax/Robust/MaxAbs/Power |
| 10 | **`X_scaled`** | M4 | (100, 42) | `m4_result.scaled_matrices['StandardScaler']` | StandardScaler selected: μ=0, σ=1 |
| 11 | **`X_reduced`** | M5 | (100, 42) | `ReductionEngine.reduce(X_scaled)` | Path 1 (None): 42→42 dims |
| 12 | `score_matrix` | L5 | (100, 26) | `L5Detection.detect_all(X_reduced)` | 26 method scores: zscore, iqr, knn, lof, IF... |
| 13 | `final_scores` | L6 | [100] | `L6Ensemble.fuse(score_matrix)` | tiered_consensus → LOW:72, NORMAL:28 |
| 14 | **`df_scored`** | L7 | (100, 80) | `df_features.copy() + scores` | FINAL: 52 + anomaly_score + risk_tier + 26 scores |

**Bold** = named DataFrame variables stored in pipeline scope.  
*(Italics)* = read-only / intermediate results not persisted as named variables.

## Phase 3 — Output & Persistence

```
pipeline.py:
  PipelineResult.df_scored = df_scored            # Returned to caller

pipeline_runner.py:
  data_vault.set_scored_data(result.df_scored)    # Persisted to vault

pipeline_service.py:
  _persist_run_data(result.df_scored, run_id)     # Parquet partition

pipeline_run.py:
  df_scored = result.df_scored                    # Used for UI charts:
    → _build_risk_by_family(df_scored)
    → _build_risk_by_ensemble(df_scored)
    → _build_scored_preview(df_scored)
```

## Complete Variable Flow Diagram

```
╔══════════════════════════════════════════════════════════════════════╗
║  DATA SOURCES PAGE (12 slots)                                      ║
╠══════════════════════════════════════════════════════════════════════╣
║                                                                    ║
║  12 CSV/Parquet   ──→  sources dict  ──→  RollupEngine             ║
║  uploads               {str: DF}          .build_master()          ║
║                                              │                     ║
║                        ┌─────────────────────┘                     ║
║                        │                                           ║
║  base_df ← BASE.copy()                                            ║
║  master_df ← base_df.dedup()                                      ║
║  master_df ← .merge(transactions_rolled)                          ║
║  master_df ← .merge(customer_party_rolled)                        ║
║  master_df ← .merge(accounts_rolled)                              ║
║  master_df ← .merge(alerts_rolled)                                ║
║  master_df ← .merge(cases_rolled)                                 ║
║  master_df ← .merge(kyc_rolled)                                   ║
║  master_df ← .merge(watchlist_rolled)                             ║
║  master_df ← .merge(others1_rolled)                               ║
║  master_df ← .merge(others2_rolled)                               ║
║       │                                                            ║
║  result.master_df = master_df                                      ║
║       │                                                            ║
║  df_master = build_result.master_df     (data_sources.py)          ║
║       │                                                            ║
║  vault/current.parquet ← set_current_data(df_master)              ║
║                                                                    ║
╠══════════════════════════════════════════════════════════════════════╣
║  PIPELINE RUN PAGE                                                 ║
╠══════════════════════════════════════════════════════════════════════╣
║                                                                    ║
║  df_master ← _load_aggregate()          (preview grid)            ║
║  df_merged ← data_vault.get_data()      (vault/current.parquet)   ║
║       │                                                            ║
║  pipeline.run(merged_df=df_merged)                                 ║
║                                                                    ║
╠══════════════════════════════════════════════════════════════════════╣
║  PIPELINE.PY — 14-Step Execution                                   ║
╠══════════════════════════════════════════════════════════════════════╣
║                                                                    ║
║  merged_df ─→ df_raw ─→ df_agg ─→ df_features                     ║
║    (input)     (L1-2)    (V6)       (L3)                           ║
║                                       │                            ║
║                          df_cleaned ←─┘                            ║
║                            (M2)                                    ║
║                              │                                     ║
║                          df_encoded                                ║
║                            (M3)                                    ║
║                              │                                     ║
║                          X_scaled                                  ║
║                            (M4)                                    ║
║                              │                                     ║
║                          X_reduced                                 ║
║                            (M5)                                    ║
║                              │                                     ║
║                          score_matrix ─→ final_scores              ║
║                            (L5)            (L6)                    ║
║                                              │                     ║
║                          df_scored = df_features.copy()            ║
║                            + anomaly_score                         ║
║                            + risk_tier                             ║
║                            + 26 score_* cols                       ║
║                            (L7 — FINAL OUTPUT)                     ║
║                                                                    ║
╠══════════════════════════════════════════════════════════════════════╣
║  OUTPUT                                                            ║
╠══════════════════════════════════════════════════════════════════════╣
║                                                                    ║
║  PipelineResult.df_scored ──→ vault (parquet)                      ║
║                           ──→ UI charts (risk by family/ensemble)  ║
║                           ──→ scored preview grid                  ║
║                                                                    ║
╚══════════════════════════════════════════════════════════════════════╝
```

## V26 Changes Applied

| # | Change | File | Before → After |
|---|--------|------|----------------|
| N1 | Version bump | config.py | 25.0.0→26.0.0, V25→V26, PORT 8101→8102 |
| N2 | Rename variable | pipeline_run.py | `agg_df` → `df_master` (9 occurrences) |
| N4 | Rename variable | pipeline_run.py | `biggest` → `df_fallback` (2 occurrences) |
| N8 | Update docstring | pipeline.py | V25→V26 header + changelog |
| N9 | Dead code marker | l3_feature_engineering.py | TODO V27 comment on `auto_transform_skewed_features()` |

### NOT changed (deferred to V27):
- N3: `master_df` inside rollup_engine.py — touches `MasterBuildResult` dataclass contract
- N5: Consolidate 5 duplicate merge paths into shared `_merge_sources_into_base()`
- N6: Fix suffix collision (`others1`/`others2` → both `_oth`)
- N7: Rename generic `df` params in `validate_base_table()`

## Known Issues for V27

### 5 Duplicate Merge Paths

| Path | File | Uses RollupEngine? | Variable |
|------|------|-------------------|----------|
| **A** (canonical) | rollup_engine.py L770 | YES (grain-aware) | `master_df` |
| B (UI wrapper) | data_sources.py L949 | YES (calls A) | `df_master` |
| **C** (simple loop) | data_sources.py L2637 | **NO** | `df_base` |
| **D** (pipeline_run) | pipeline_run.py L540 | **NO** | `df_base` → `df_merged` |
| **E** (L1 ingest) | l1_l2_ingestion.py L230 | Partial | `self.base_df` |

**Paths C and D join raw tables with NO grain normalization** — they should delegate to Path A.

### Suffix Collision
`_{table_name[:3]}` makes `others1` and `others2` both get suffix `_oth`.  
Fix: use `_{table_name[:6]}` or full name.

### Dead Code
`auto_transform_skewed_features()` in l3_feature_engineering.py — implemented but never called.

In [ ]:
# Quick verification: load V26 config and confirm version
import sys
sys.path.insert(0, '.')
from config import APP
print(f"Version:  {APP.VERSION}")
print(f"Tag:      {APP.VERSION_TAG}")
print(f"Port:     {APP.PORT}")
print(f"Title:    {APP.TITLE}")